### Week 1. Titanic Classification with DistilBERT + XGBoost 과제

Week 1. 과제로는 `week1_lecture.ipynb` 코드 예제를 참고하여, `kaggle`에서 titanic 데이터를 받아 데이터 전처리를 진행한 후 학습해 자신의 모델을 만든 후, ipynb 파일과 결과 csv파일을

- ***hw1_학번_이름.ipynb***
- ***submission.csv***

실전데이터사이언스 스터디 repository week1 폴더에 커밋하시면 됩니다.!

https://github.com/a2ran/prac_ds

타이타닉 데이터 전처리 예시는 다음과 같습니다.

1. Filling out missing (NaN) Values
2. VIF으로 분산이 높은 column 제거 or 수정
3. train_test_split 으로 train/val 나누는 과정 수정
4. "bert-base-uncased" 이외 다른 BERT 모델을 사용해 Embedding하기
5. "xgboost" 이외 다른 머신러닝 알고리즘 사용

참고할만한 example은 다음과 같습니다.


https://github.com/minsuk-heo/kaggle-titanic/blob/master/titanic-solution.ipynb

`Optional` :

https://github.com/mrdbourke/your-first-kaggle-submission/blob/master/kaggle-titanic-dataset-example-submission-workflow.ipynb
https://github.com/agconti/kaggle-titanic/blob/master/Titanic.ipynb


# 기본 제공 코드

### 1. Kaggle에서 데이터 받아오기

In [ ]:
## pip install ~ : 패키지 다운로드
## -q : 로그 메세지 출력 X

!pip install -q kaggle

In [ ]:
## Kaggle에서 데이터를 받아오기 위해서는 Authenticator Token인 "kaggle.json"이 있어야 합니다.
## 자세한 내용은 영상을 참고하세요

from google.colab import drive
drive.mount("/content/drive")

!mkdir ~/.kaggle
## Drive에 kaggle.json을 업로드한 경로를 적으시면 됩니다. ex) (/content/drive/MyDrive/study_session/kaggle.json)
!cp /content/drive/MyDrive/데싸랩/Seminar/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive


In [ ]:
# 공모전 이름
competition_name = "titanic"

# 공모전 다운로드 to local environment
! kaggle competitions download -c {competition_name}

# {competition_name}이름의 폴더에 zip 파일 압축해제
! unzip {competition_name + ".zip"} -d {competition_name}

# 드라이브 확인을 완료했으므로 드라이브 mount를 해제합니다.
drive.flush_and_unmount()

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 31.5MB/s]
Archive:  titanic.zip
  inflating: titanic/gender_submission.csv  
  inflating: titanic/test.csv        
  inflating: titanic/train.csv       


### 2. 데이터 전처리

In [ ]:
# 타이타닉 데이터 전처리 예시는 다음과 같습니다.

# Filling out missing (NaN) Values
# VIF으로 분산이 높은 column 제거 or 수정
# train_test_split 으로 train/val 나누는 과정 수정
# "bert-base-uncased" 이외 다른 BERT 모델을 사용해 Embedding하기
# "xgboost" 이외 다른 머신러닝 알고리즘 사용

### 결측치 채우기

In [ ]:
## GPU 활성화
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import numpy as np
import pandas as pd

train = pd.read_csv('titanic/train.csv')
test = pd.read_csv('titanic/test.csv')

In [ ]:
## Your Code
# train.info() #age에 177개, cabin에 687개, embarked에 2개


In [ ]:
# Cabin 채우기


## 결측된 Cabin 값에 넣을만한 값 찾아보기
# df1 = train[~train['Cabin'].isnull()].sort_values(by = 'Pclass')
# import re
# def extract_letters(text):
#     letter = re.sub("[^a-zA-Z]", "", text)
#     return letter

# # 특정 열에서 문자만 추출
# df1['letters'] = df1['Cabin'].apply(extract_letters)
# df1['letters'] = df1['letters'].apply(lambda x: x[0] if len(x) >= 2 else x)
# df1['letters'] = df1['letters'].map({'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': '7'})
# df1 = df1[~df1['letters'].isna()] # T 객실 제외

# df1['letters'] = df1['letters'].astype(int)
# df1.groupby('Pclass')['letters'].mean()
## 1등급은 평균/최빈 C/C, 2등급은 E/F, 3등급은 F/F
## Cabin에서 1등급은 C, 2등급은E, 3등급은 F로 채우고, 뒤의 숫자는 00으로 하도록 함.
## 이렇게 결측치 처리한 케이스는 Unknown1에 1로 표기

train.loc[(train['Cabin'].isnull()) & (train['Pclass'] == 1), 'Cabin'] = 'C00'
train.loc[(train['Cabin'].isnull()) & (train['Pclass'] == 2), 'Cabin'] = 'E00'
train.loc[(train['Cabin'].isnull()) & (train['Pclass'] == 3), 'Cabin'] = 'F00'
train['unknown_1'] = 0
train.loc[train['Cabin'].isin(['C00', 'E00', 'F00']), 'unknown_1'] = 1
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,unknown_1
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,F00,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,F00,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,F00,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,E00,S,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,F00,S,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,0


In [ ]:
### embarked 채우기 ###
# train.iloc[50:80]
# train.iloc[810:840]
## 승객 id 순서는 embarked에 영향 X
# train[train['Embarked'].isna()] # 둘 다 Cabin이 2
# train[train['Cabin'].str.contains('B')]['Embarked'].mode() #B동의 승객은 Embarked = S가 최빈값.

train.loc[train['Embarked'].isna(), 'Embarked'] = 'S'


In [ ]:
### Age 채우기 ###
# SibSq, Parch 유형을 4가지로 구분하고 그에 따라 랜덤으로 나이 입력


train['unknown_2'] = 0
df1 = train[~train['Age'].isna()]

# 혼자
age_list = df1.loc[(df1['Parch'] == 0) & (df1['SibSp'] == 0)]['Age']
mask = (train['Parch'] == 0) & (train['SibSp'] == 0) & (train['Age'].isnull())
selected_rows = train.loc[mask]
random_values = np.random.choice(age_list, size=len(selected_rows))
train.loc[mask, 'unknown_2'] = 1
train.loc[mask, 'Age'] = random_values

# 부모자식 X, 형제 O
age_list= df1.loc[(df1['Parch'] != 0) & (df1['SibSp'] == 0)]['Age']
mask = (train['Parch'] != 0) & (train['SibSp'] == 0) & (train['Age'].isnull())
selected_rows = train.loc[mask]
random_values = np.random.choice(age_list, size=len(selected_rows))
train.loc[mask, 'unknown_2'] = 1
train.loc[mask, 'Age'] = random_values

# 부모자식 O, 형제 X
age_list = df1.loc[(df1['Parch'] == 0) & (df1['SibSp'] != 0)]['Age']
mask = (train['Parch'] == 0) & (train['SibSp'] != 0) & (train['Age'].isnull())
selected_rows = train.loc[mask]
random_values = np.random.choice(age_list, size=len(selected_rows))
train.loc[mask, 'unknown_2'] = 1
train.loc[mask, 'Age'] = random_values

# 부모 형제 모두
age_list = df1.loc[(df1['Parch'] != 0) & (df1['SibSp'] != 0)]['Age']
mask = (train['Parch'] != 0) & (train['SibSp'] != 0) & (train['Age'].isnull())
selected_rows = train.loc[mask]
random_values = np.random.choice(age_list, size=len(selected_rows))
train.loc[mask, 'unknown_2'] = 1
train.loc[mask, 'Age'] = random_values



In [ ]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,unknown_1,unknown_2
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,F00,S,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,F00,S,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,F00,S,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,E00,S,1,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,7.0,1,2,W./C. 6607,23.4500,F00,S,1,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,0,0


### VIF

In [ ]:
train['Sex'] = train['Sex'].map({'male' : 0, 'female' : 1})
train['Embarked'] = train['Embarked'].map({'S' : 0, 'C' : 1, 'Q' : 3})
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,unknown_1,unknown_2
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,F00,0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,F00,0,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,0,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,F00,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",0,27.0,0,0,211536,13.0000,E00,0,1,0
887,888,1,1,"Graham, Miss. Margaret Edith",1,19.0,0,0,112053,30.0000,B42,0,0,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,7.0,1,2,W./C. 6607,23.4500,F00,0,1,1
889,890,1,1,"Behr, Mr. Karl Howell",0,26.0,0,0,111369,30.0000,C148,1,0,0


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF 계산할 변수 선택 및 수치형 변환
selected_variables = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
train[selected_variables] = train[selected_variables].apply(pd.to_numeric, errors='coerce')

# VIF 계산을 위한 독립 변수 행렬 X 준비
X = train[selected_variables]

# 각 변수의 VIF 계산
vif = pd.DataFrame()
vif["Variable"] = selected_variables
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif
# vfi가 10 이상인 값이 없음

### train test로 자르기

In [ ]:
from sklearn.model_selection import train_test_split

X = train.drop(columns = ['PassengerId', 'Survived'], axis = 1)
y = train['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state = 77, stratify = y)

### 3. 데이터 임베딩

In [ ]:
from typing import List
from tqdm.notebook import tqdm

!pip install -q transformers
from transformers import AutoModel, AutoTokenizer

class Encode_with_RoBERTa:
    def __init__(self):
        ## Huggingface에서 RoBERTa 모델을 받아옵니다.
        self.tokenizer = AutoTokenizer.from_pretrained('roberta-base')
        self.model = AutoModel.from_pretrained('roberta-base').to(device)

    ## 입력한 `texts`을 768 길이의 숫자벡터로 특징을 추출합니다.
    def extract(self, texts: List[str]):
        features = np.zeros((len(texts), 768), dtype = np.float16)
        for index, text in enumerate(tqdm(texts)):
            tokenized_text = self.tokenizer(text, return_tensors="pt").to(device)
            model_output = self.model(**tokenized_text)[0].detach().cpu()
            features[index, :] = model_output.numpy().mean(axis=1)

        return features

### 4. 임베딩한 데이터 머신러닝 알고리즘으로 분류

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

extractor = Encode_with_RoBERTa()
scaler = StandardScaler()
classifier = AdaBoostClassifier()

## AdaBoost 학습
train_labels = [int(_) for _ in y_train.values] ## [0,1,1, ...]
texts = [", ".join(str(_)) for _ in X_train.values] ##[[0,1,male],[2,1,female]...]
train_features = scaler.fit_transform(extractor.extract(texts)) ## to (len(texts), 768) 숫자벡터
classifier.fit(train_features, train_labels) ## XGBoost train

## Prediction with Test Data
answer = [int(_) for _ in y_test.values]
texts = [", ".join(str(_)) for _ in X_test.values]
preds = classifier.predict(scaler.transform(extractor.extract(texts)))

## 모델 예측의 정밀도 측정!
accuracy = accuracy_score(answer, preds)
print(f'\naccuracy : {accuracy*100:.2f}%')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/846 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]


accuracy : 77.78%


### 5. csv로 저장 + kaggle에 submit

In [ ]:
import csv

ids = test['PassengerId'].values
texts = [", ".join(str(_)) for _ in test.iloc[:, 1:].values]
preds = classifier.predict(scaler.transform(extractor.extract(texts)))

with open('submission.csv', "w") as to_file:
    csvwriter = csv.writer(to_file, delimiter=",", quotechar='"')
    csvwriter.writerow(["PassengerId", "Survived"])
    for id, pred in zip(ids, preds):
        csvwriter.writerow([id, pred])

  0%|          | 0/418 [00:00<?, ?it/s]

In [ ]:
sub = pd.read_csv('submission.csv')
sub.head(5)

,PassengerId,Survived
0,892,1
1,893,1
2,894,1
3,895,1
4,896,1
